In [60]:
import os
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_core.tools import Tool
from langchain.chains import LLMMathChain

In [61]:
key_file='../openai_api_key.txt'
with open(key_file, 'r') as file:
    openai_api_key=file.read()
os.environ["OPENAI_API_KEY"] = openai_api_key

In [62]:
verbose=False

# This simple prompt simulates the case of accessing private information.
concealedinfo_prompt = PromptTemplate.from_template("Mary is 24. Given this information, answer the following question: {question}")
concealedinfo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=concealedinfo_prompt, verbose=verbose)

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

calculate_chain = LLMMathChain.from_llm(llm=OpenAI(temperature=0), verbose=verbose) 

tools=[ wiki_tool,
        Tool(
            name="CALCULATE",
            func=calculate_chain.run, 
            description="Useful for making math calculations. Input: a calculation to perform on a set of numbers (these need to be numbers, not literal expressions). Outputs: the result of the calculation."
        ),
        Tool(
            name="INFO_ABOUT_MARY",
            func=concealedinfo_chain.run,
            description='Only needed to answer a question that requires knowledge about Mary. Do not use unless Mary is involved.' 
        ),
]


LLM temperature will be set to a non-zero value in order to have slightly different responses later on.

In [63]:

prompt = hub.pull("hwchase17/openai-functions-agent") # To see the prompt in full: https://smith.langchain.com/hub/hwchase17/openai-functions-agent

llm = ChatOpenAI(model='gpt-3.5-turbo-1106',temperature=0.1)

agent = create_openai_functions_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=verbose, return_intermediate_steps=True, 
)

We will invoke the agent different times in order to verify the robustness of its responses.

In [64]:
response = agent_executor.invoke({"input": "What is the sum of the ages of Mary and of Leonardo Di Caprio (as of 2023)?"},    
                                     config={"configurable": {"session_id": "<foo>"}},
    )

In [65]:
for i in range(5):
    response = agent_executor.invoke({"input": "What is the sum of the ages of Mary and of Leonardo Di Caprio (as of 2023)?"})
    answer=response['output']
    print(f"{answer=}")
    if '73' not in answer:
        print(response['intermediate_steps'])
        wrong_answer=answer

answer='The sum of the ages of Mary and Leonardo Di Caprio (as of 2023) is 72.'
[(AgentActionMessageLog(tool='INFO_ABOUT_MARY', tool_input='age', log='\nInvoking: `INFO_ABOUT_MARY` with `age`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"age"}', 'name': 'INFO_ABOUT_MARY'}})]), '\n\n24'), (AgentActionMessageLog(tool='wikipedia', tool_input='Leonardo Di Caprio', log='\nInvoking: `wikipedia` with `Leonardo Di Caprio`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"Leonardo Di Caprio"}', 'name': 'wikipedia'}})]), 'Page: Leonardo DiCaprio\nSummary: Leonardo Wilhelm DiCaprio (; Italian: [diˈkaːprjo]; born November 1'), (AgentActionMessageLog(tool='CALCULATE', tool_input='24 + 48', log='\nInvoking: `CALCULATE` with `24 + 48`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"24 + 48"}', 'name': 'CALCULATE'}})])

We expect the answer to be 24+49=73. Any answer that does not contain 71 will make the code print the intermediate steps for inspection.

Let's now provide the agent with memory, and let's use it to evaluate the wrong answer we got.

In [66]:

from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

memory = ChatMessageHistory(session_id="test-session")

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [71]:

memory.add_user_message("What is the sum of the ages of Mary and of Leonardo Di Caprio (as of 2023)?")
memory.add_user_message(wrong_answer)

response = agent_with_chat_history.invoke({"input": "Are you sure about DiCaprio's age?"},
                                     config={"configurable": {"session_id": "<foo>"}}
                                     )
print(response['chat_history'])
print(response['output'])

memory.messages=[] # We finally clear the memory

[HumanMessage(content='What is the sum of the ages of Mary and of Leonardo Di Caprio (as of 2023)?'), HumanMessage(content='I have found that Mary is 24 years old. Leonardo DiCaprio was born on November 11, 1974. As of 2023, he would be 48 years old. \n\nTherefore, the sum of the ages of Mary and Leonardo DiCaprio as of 2023 would be 24 + 48 = 72.')]
I apologize for the error. Leonardo DiCaprio was born on November 11, 1974. As of 2023, he would be 49 years old. 

Therefore, the sum of the ages of Mary and Leonardo DiCaprio as of 2023 would be 24 + 49 = 73.
